# Objective
Get comfortable with git-objects. Tutorial adapted heavily from [here](https://book.git-scm.com/book/en/v2/Git-Internals-Git-Objects).

At the end of this, you should be able to answer:
1. What is a blob?
2. What is a tree?
3. What is a commit?

You should be able to describe the lower-level procedures, that underpin:
- `git add`
- `git commit -m`

We will also cover `git stash`.

First, let's check what working directory we are in.

In [ ]:
%%bash
echo $PWD

We set `/home/jovyan/notebooks/mock_repo` as the default directory.

In [ ]:
%cd /home/jovyan/notebooks/mock_repo

In [ ]:
%%bash
echo $PWD

# Creating git object out of thin air

Following comment pipes 'test content' to the command `git hash-object -w --stdin`. (The `|` in command line is a pipe.)

This creates a Git object, that we can see using the command `find .git/objects -type f` or just by cd-ing to the `.git/objects` directory and taking a look ourselves.

In [ ]:
%%bash
echo 'test content' | git hash-object -w --stdin

In [ ]:
%%bash
cd .git/objects
ls -la

Ah, it looks like there is a sub-diretory called `d6`. This is where objects with hashes starting with 'd6' go to. Let's check out this directory.

In [ ]:
%%bash
cd .git/objects/d6
ls -la

In [ ]:
%%bash
cd .git/objects/d6
ls -la | awk 'FNR == 4{a="d6"$NF; print a}'

**git cat-file**<br>
So we see the output of the hash. We feed this command into `git cat-file`. What `git cat-file` does, is to take the hash of a git object, and print its contents. `-p` is a flag specifying pretty-print, and allows us to cat-file without setting the object type.

An alternative command without the flag `-p` is:
`git cat-file blob d670460b4b4aece5915caf5c68d12f560a9fe3e4`

**awk**<br>
As an aide, awk is a useful tool to parse text.
- `FNR` stands for record number (i.e. the line) in the current file.
- `NF` is the number of fields, where fields represent the columns. So, `$NF` prints the 9th column of the `ls -la` output. If we want the 8th column, we go with `$8`
- So, we are taking the 4th record (i.e. the 4th line), and the last field of that line.

In [ ]:
%%bash
cd .git/objects/d6
git cat-file -p $(ls -la | awk 'FNR == 4{a="d6"$NF; print a}')

This is the simpler way, involving just copy-and-pasting. But it's useful to know your way around the command line.

In [ ]:
%%bash
git cat-file -p d670460b4b4aece5915caf5c68d12f560a9fe3e4

Taking a step back, we see that our working directory is empty. But we have a blob in `.git/objects/`. Why is this the case?

In [ ]:
%%bash
echo $PWD
ls -la

What we have done is indeed unusual. We didn't create a file in the working directory, but we created a blob (a type of Git object), by piping a stream of text into the command `git hash-object`.

What happens in the more common `git add`, is that prior to including the file in the index (i.e. the file is still in the working directory), the file is not a in the directory `.git/objects/`. But the moment we add the file to the index, the file content exists in two places: the working directory, and also in `.git/objects/` as a hashed object whose contents are revealed with `git cat-file`.